In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# -----------------------------
# 1) Load & prepare the data
# -----------------------------
csv_path = "jeroen_punt_nl_dynamische_stroomprijzen_jaar_2024.csv"

# auto-detect ; or , as separator; parse Dutch decimals with comma
df = pd.read_csv(csv_path, sep=None, engine="python", decimal=",")

# use local NL timestamp
df["datum_nl"] = pd.to_datetime(df["datum_nl"])
df = df.sort_values("datum_nl")

# ensure the price is numeric (€/kWh)
df["prijs_excl_belastingen"] = pd.to_numeric(df["prijs_excl_belastingen"], errors="coerce")

# optional smooth reference (24h rolling mean)
df["prijs_24h_mean"] = df["prijs_excl_belastingen"].rolling(24, min_periods=1).mean()

# -----------------------------
# 2) Fixed-price reference levels (ALL shown in the LEGEND)
# -----------------------------
FIX_LOW  = 0.13   # €/kWh  (laag)
FIX_AVG  = 0.18   # €/kWh  (gemiddeld)
FIX_HIGH = 0.22   # €/kWh  (hoog)

# -----------------------------
# 3) Plotly Express line figure (dynamic + 24h mean)
# -----------------------------
fig = px.line(
    df,
    x="datum_nl",
    y=["prijs_excl_belastingen", "prijs_24h_mean"],
    labels={"datum_nl": "Date", "value": "Price (€/kWh)", "variable": ""},
    title="Dynamic hourly electricity price – 2024 (incl. 24h mean & fixed-price references)",
    template="plotly_white"
)

# nicer hover + ranges
fig.update_layout(
    hovermode="x unified",
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=7, step="day", stepmode="backward", label="7d"),
                dict(count=1, step="month", stepmode="backward", label="1m"),
                dict(count=3, step="month", stepmode="backward", label="3m"),
                dict(count=6, step="month", stepmode="backward", label="6m"),
                dict(step="all", label="All"),
            ]
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)
fig.update_yaxes(tickprefix="€", tickformat=".3f")
fig.update_traces(hovertemplate="%{x|%Y-%m-%d %H:%M}<br>€%{y:.3f}")

# -----------------------------
# 4) Add fixed-price reference LINES as real traces (so they appear in the LEGEND)
# -----------------------------
x0, x1 = df["datum_nl"].min(), df["datum_nl"].max()

for y, name, dash in [
    (FIX_LOW,  f"Low fixed €{FIX_LOW:.2f}/kWh",  "dot"),
    (FIX_AVG,  f"Avg fixed €{FIX_AVG:.2f}/kWh",  "dash"),
    (FIX_HIGH, f"High fixed €{FIX_HIGH:.2f}/kWh", "dashdot"),
]:
    fig.add_trace(
        go.Scatter(
            x=[x0, x1], y=[y, y],
            mode="lines",
            name=name,
            line=dict(dash=dash, width=2),
            hoverinfo="skip",        # keep hover clean on the main series
            showlegend=True
        )
    )

# If you prefer only the raw hourly line, hide the rolling mean:
# fig.for_each_trace(lambda tr: tr.update(visible="legendonly") if tr.name=="prijs_24h_mean" else ())

fig.show()
